1/ Importation des données TC et TITMC 

2/ Import et traitement du fichier des prénoms 

3/ Ajouter le genre des personnes 

4/ Codification des adresses 

5/ Ajout et traitement du code commune

6/ Retraiter et simplifier les qualités 

In [1]:
# Lecture de la base inpi
import os
import s3fs
import pandas as pd
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "radjerad/inpi"
FILE_KEY_S3 = "data_rep_pm_2017.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df_rep_pm = pd.read_csv(file_in, sep=";", encoding="utf-8")

In [2]:
df_rep_pm.shape # 9 751 833

(9751833, 73)

In [3]:
df_rep_pm = df_rep_pm.rename({"code greffe": "code_greffe"}, axis=1)

In [4]:
df_rep_pm = df_rep_pm.loc[:, ["code_greffe", "siren", "denomination", "forme_juridique_x", "qualité",
"nom_patronymique", "nom_usage", "prénoms", "type", "date_naissance", "ville_naissance", "adresse_ligne1", "adresse_ligne2",
"adresse_ligne3", "code_postal", "ville", "code_commune", "pays", "id_représentant"]]

In [5]:
df_rep_pm = df_rep_pm.rename({"forme_juridique_x": "forme_juridique", "prénoms": "prenoms", "qualité": "qualite", "id_représentant": "id_representant"}, axis=1)

In [6]:
df_rep_pm.head(10)

,code_greffe,siren,denomination,forme_juridique,qualite,nom_patronymique,nom_usage,prenoms,type,date_naissance,ville_naissance,adresse_ligne1,adresse_ligne2,adresse_ligne3,code_postal,ville,code_commune,pays,id_representant
0,101,15850944,AGIR TECHNOLOGIES,Société par actions simplifiée à associé unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101,15851793,DORAS,Société anonyme à directoire et conseil de sur...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Président,VATIN,NaN,Gerard,P.Physique,1947-10-07,79270 Saint-Maixent-L'École 79,NaN,1360 chemin Saint-Roch,NaN,84200.0,Carpentras,84031.0,FRANCE,1.0
3,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,VATIN,NaN,Jacques,P.Physique,1930-04-03,02691 Saint-Quentin 02,NaN,50 rue de la Tour,NaN,75016.0,Paris,NaN,FRANCE,3.0
4,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,JORDENS,VATIN,Regina,P.Physique,1948-08-07,Allemagne,NaN,1360 chemin Saint-Roch,NaN,84200.0,Carpentras,84031.0,FRANCE,4.0
5,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,VATIN,VATIN-CHAIX,Bertrand Heinrich René,P.Physique,1977-06-09,Chêne bougeries,NaN,63 rue Virgile,NaN,67200.0,Strasbourg,67482.0,FRANCE,11.0
6,101,54500335,SOCIETE NATIONALE DES ETABLISSEMENTS PIOT PNEU,Société anonyme à directoire et conseil de sur...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,101,54800958,RICHARDSON,Société par actions simplifiée,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,101,56502248,SAMSE,Société anonyme à directoire et conseil de sur...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,101,56503097,BMRA,Société par actions simplifiée,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_rep_pm["qualite"].isna().sum() # 246 926

246926

In [8]:
# Traitement des cas où le nom et les prénoms sont manquants


In [6]:
df_rep_pm.loc[df_rep_pm["prenoms"] == "-", "prenoms"] = ""

In [7]:
df_rep_pm.loc[df_rep_pm["prenoms"] == "", :].shape # 4 448

(4448, 19)

In [11]:
df_rep_pm.loc[(df_rep_pm["prenoms"] == "") & (df_rep_pm["type"] == "P.Physique"), :].shape # 4 448

(4448, 19)

In [8]:
re1 = "(INDIVISION SUCCESSORALE DE M\.\s|\sM\.\s|^M\.\s|MME\.\s|MR\.\s|MELLE\.\s|MONSIEUR\.?\s|MADAME\.?\s|MLLE\.\s|REPRESENTEE\sPAR\sMONSIEUR\.?\s|REPRESENTEE\sPAR\s)"
df_rep_pm["top_civilite"] = df_rep_pm["nom_patronymique"].str.contains(re1, regex=True)

/tmp/ipykernel_1341/3266536503.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_rep_pm["top_civilite"] = df_rep_pm["nom_patronymique"].str.contains(re1, regex=True)


In [ ]:
df_rep_pm.loc[df_rep_pm["top_civilite"] == True, "nom_patronymique"].head()

In [ ]:
#df_rep_pm["contenu_civilite"] = df_rep_pm["nom_patronymique"].str.extract(re1)

In [ ]:
#df_rep_pm.loc[df_rep_pm["top_civilite"] == True, "contenu_civilite"].head()

In [9]:
df_rep_pm["nom_patronymique2"] = df_rep_pm["nom_patronymique"]

In [10]:
df_rep_pm["nom_patronymique2"] = df_rep_pm["nom_patronymique2"].str.replace(re1,"")

/tmp/ipykernel_1341/259944700.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_rep_pm["nom_patronymique2"] = df_rep_pm["nom_patronymique2"].str.replace(re1,"")


In [15]:
from pandas import option_context
with option_context('display.max_colwidth', 400):
    print(df_rep_pm.loc[df_rep_pm["top_civilite"] == True, "nom_patronymique2"].head(10))

151606                               DE VILLARDI DE MONTLAUR
158708                                 DE LAITRE JEAN PIERRE
320046                            C.I.I.M. LAVIGUEUR BERNARD
320809    SARL S.A.F. SA GERANTE LABARTHE NEE CALITRO SABINE
322994             SARL EURO DELTA IMMOBILIER CULLET CORINNE
322995             SARL EURO DELTA IMMOBILIER CULLET CORINNE
323752                                     TALAL YOUSUFZAHID
331301                                       MARCELLIER RENE
331501                                                 PINCE
360161                               DURANTI PIERRE FRANCOIS
Name: nom_patronymique2, dtype: object


In [11]:
# on repartit les noms et les prénoms dans les bonnes variables
# on a modifié un peu le code de l'insee pour enlever les representee par, ici aussi car on va essayer de prendre en compte les particules
particule = "(DU\s[A-Za-z]+|DE\sLA\s[A-Za-z]+|[A-Za-z]+\sDU\s[A-Za-z]+|[A-Za-z]+\sDE\sLA\s[A-Za-z]+|DE\s[A-Za-z]+|[A-Za-z]+\sDE\s[A-Za-z]+)"
#particule = "(DU\s|DE\sLA\s|\sDU\s|\sDE\sLA\s|DE\s|\sDE\s)"
df_rep_pm["top_particule"] = df_rep_pm["nom_patronymique"].str.contains(particule, regex=True)

/tmp/ipykernel_1341/837695662.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_rep_pm["top_particule"] = df_rep_pm["nom_patronymique"].str.contains(particule, regex=True)


In [ ]:
df_rep_pm.loc[(df_rep_pm["prenoms"] == "") | (df_rep_pm["prenoms"].isna()),:].shape # 1093022

In [ ]:
df_rep_pm.loc[(df_rep_pm["prenoms"] == ""),:].shape # 4448, mais ce sont tous des cas différents, donc on n'a pas envie de séparer nom et prénoms comme le fait l'insee

In [ ]:
df_rep_pm.loc[(df_rep_pm["top_particule"] == True) & (df_rep_pm["prenoms"] == ""), "nom_patronymique2"].head(10)

In [ ]:
df_rep_pm.loc[df_rep_pm["nom_usage"].isna(), :].head() # pas d'info parfois quand il y a l'info nom patronymique et prénoms 

In [16]:
# nettoyage de la variable qualite 
df_rep_pm["qualite"].value_counts()

Gérant                                  2413218
Associé                                 2174834
Gérant                                  1112522
Associé indéfiniment responsable        1089112
gérant                                   633900
                                         ...   
Adhérent                                      1
Directeur technique et administratif          1
Comptable                                     1
Contrôleur des comptes adjoint                1
Mandataire général                            1
Name: qualite, Length: 371, dtype: int64

In [12]:
df_rep_pm["qualite2"] = df_rep_pm["qualite"]

In [13]:
df_rep_pm["qualite2"] = df_rep_pm["qualite2"].str.lower()
df_rep_pm["qualite2"] = df_rep_pm["qualite2"].str.strip()

In [19]:
df_rep_pm["qualite2"].value_counts()

gérant                                                                                                              4159640
associé                                                                                                             2174857
associé indéfiniment responsable                                                                                    1089112
président                                                                                                            578505
commissaire aux comptes titulaire                                                                                    249159
                                                                                                                     ...   
adhérent                                                                                                                  1
directeur technique et administratif                                                                                      1
membre t

In [14]:
# nettoyage de la dénomination
df_rep_pm["denomination2"] = df_rep_pm["denomination"]

In [15]:
df_rep_pm["denomination2"] = df_rep_pm["denomination2"].str.lower()

In [22]:
df_rep_pm["denomination2"].head(10)

0                                    agir technologies
1                                                doras
2    societe immobiliere financiere d etudes et de ...
3    societe immobiliere financiere d etudes et de ...
4    societe immobiliere financiere d etudes et de ...
5    societe immobiliere financiere d etudes et de ...
6       societe nationale des etablissements piot pneu
7                                           richardson
8                                                samse
9                                                 bmra
Name: denomination2, dtype: object

In [ ]:
# codification des adresses
df_rep_pm.columns

In [23]:
df_rep_pm["ville"].head(10)

0           NaN
1           NaN
2    Carpentras
3         Paris
4    Carpentras
5    Strasbourg
6           NaN
7           NaN
8           NaN
9           NaN
Name: ville, dtype: object

In [24]:
df_rep_pm.loc[df_rep_pm["ville"] == "", :].shape

(0, 23)

In [25]:
df_rep_pm.loc[(df_rep_pm["ville"].isna()) & (df_rep_pm["adresse_ligne3"].isna() == False), :].head(10) #90740

,code_greffe,siren,denomination,forme_juridique,qualite,nom_patronymique,nom_usage,prenoms,type,date_naissance,...,adresse_ligne3,code_postal,ville,code_commune,pays,id_representant,top_civilite,nom_patronymique2,qualite2,denomination2
27,101,73500050,SARL DUMAN,Société à responsabilité limitée,Gérant,BORODAGE,NaN,Vilem Pierre,P.Physique,1945-12-27,...,BELLEY,1300.0,NaN,NaN,FRANCE,1.0,False,BORODAGE,gérant,sarl duman
78,101,300434446,TRANCHAND,Société par actions simplifiée,Directeur général,TRANCHAND,NaN,Claude,P.Physique,1937-11-12,...,Belley,1300.0,NaN,NaN,FRANCE,1.0,False,TRANCHAND,directeur général,tranchand
239,101,301296075,SOCIETE JURASSIENNE DE CONSTRUCTION,Société à responsabilité limitée,Gérant,DAVID,SCHAPOSCHNIKOFF,Alice Henriette,P.Physique,1930-01-28,...,NEUILLY SUR SEINE,NaN,NaN,NaN,FRANCE,1.0,False,DAVID,gérant,societe jurassienne de construction
256,101,301524302,ENTREPRISE PETETIN,Société à responsabilité limitée,Gérant,PETETIN,NaN,Jean-Yves Raoul,P.Physique,1965-05-01,...,01000 Saint-Denis-Les-Bourg,NaN,NaN,NaN,FRANCE,1.0,False,PETETIN,gérant,entreprise petetin
278,101,301899522,ANDRE NEYRET IMMOBILIER,Société à responsabilité limitée,Gérant,NEYRET,NaN,André,P.Physique,1946-11-26,...,Chalamont,NaN,NaN,NaN,FRANCE,1.0,False,NEYRET,gérant,andre neyret immobilier
303,101,302329420,MACDERMID FRANCE,Société par actions simplifiée à associé unique,Président,TORNE SANCHO,NaN,Javier,P.Physique,1975-05-18,...,Barcelone,NaN,NaN,NaN,ESPAGNE,26.0,False,TORNE SANCHO,président,macdermid france
314,101,302458344,STYL MONDE SA,Société anonyme à directoire et conseil de sur...,Membre du directoire,MOREAU,NaN,Bernard Jean-Paul,P.Physique,1961-11-20,...,Chazey-sur-Ain 01,NaN,NaN,NaN,FRANCE,8.0,False,MOREAU,membre du directoire,styl monde sa
394,101,302725197,SCI DU BALZAC,Société civile immobilière,Associé indéfiniment responsable,TURRETTINI,NaN,Benedict Francois,P.Physique,1944-10-16,...,Vesenaz CH 1222,NaN,NaN,NaN,SUISSE,1.0,False,TURRETTINI,associé indéfiniment responsable,sci du balzac
395,101,302725197,SCI DU BALZAC,Société civile immobilière,Gérant,TURRETTINI,NaN,Benedict Francois,P.Physique,1944-10-16,...,Vesenaz CH 1222,NaN,NaN,NaN,SUISSE,1.0,False,TURRETTINI,gérant,sci du balzac
400,101,303242408,CEDEC SA,Société de droit étranger,Président du conseil d'administration,NICLAUS,NaN,Jean Émile,P.Physique,1930-01-26,...,98000 Monaco,NaN,NaN,NaN,MONACO,2.0,False,NICLAUS,président du conseil d'administration,cedec sa


In [16]:
df_rep_pm.loc[(df_rep_pm["ville"].isna()) & (df_rep_pm["adresse_ligne3"].isna() == False), "ville"] = \
    df_rep_pm.loc[(df_rep_pm["ville"].isna()) & (df_rep_pm["adresse_ligne3"].isna() == False), "adresse_ligne3"]

In [56]:
df_rep_pm.columns

Index(['code_greffe', 'siren', 'denomination', 'forme_juridique', 'qualite',
       'nom_patronymique', 'nom_usage', 'prenoms', 'type', 'date_naissance',
       'ville_naissance', 'adresse_ligne1', 'adresse_ligne2', 'adresse_ligne3',
       'code_postal', 'ville', 'code_commune', 'pays', 'id_representant',
       'top_civilite', 'nom_patronymique2', 'top_particule', 'qualite2',
       'denomination2', 'ville2', 'code_postal2', 'code_departement'],
      dtype='object')

In [57]:
df_rep_pm["commune"] = df_rep_pm["ville"]

In [58]:
df_rep_pm["commune"] = df_rep_pm["commune"].str.lower()

In [ ]:
df_rep_pm["commune"].head(10)

In [59]:
df_rep_pm["commune"] = (df_rep_pm["commune"].str.replace("à", "a")
                                            .str.replace("è", "e")
                                            .str.replace("é", "e")
                                            .str.replace("ç", "c")
                                            .str.replace("ù", "u")
                                            .str.replace("ô", "o"))

In [60]:
df_rep_pm["commune"] = (df_rep_pm["commune"].str.replace("SAINT", "ST"))

In [61]:
df_rep_pm["commune"] = df_rep_pm["commune"].str.replace(" ", "")

In [62]:
df_rep_pm["commune"].value_counts()

paris                720512
marseille            134835
lyon                 104458
toulouse              75744
nice                  73523
                      ...  
aubort                    1
pontchara                 1
danpierresurblevy         1
1246corsiersuisse         1
56283beulich              1
Name: commune, Length: 133145, dtype: int64

In [38]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "radjerad/diffusion/code_postal"
FILE_KEY_S3 = "laposte_hexasmal.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    code_postal = pd.read_csv(file_in, sep=";")

In [21]:
code_postal.head()

,Code_commune_INSEE,Nom_commune,Code_postal,Ligne_5,Libellé_d_acheminement,coordonnees_gps
0,25620,VILLE DU PONT,25650,NaN,VILLE DU PONT,"46.999873398,6.498147193"
1,25624,VILLERS GRELOT,25640,NaN,VILLERS GRELOT,"47.361512085,6.235167025"
2,26001,SOLAURE EN DIOIS,26150,AIX EN DIOIS,SOLAURE EN DIOIS,"44.70587354,5.387770455"
3,26003,ALEYRAC,26770,NaN,ALEYRAC,"44.499150841,4.946083614"
4,26009,ANDANCETTE,26140,NaN,ANDANCETTE,"45.246625023,4.812717032"


In [34]:
code_postal.shape # 39 193

(39193, 6)

In [35]:
df_rep_pm.loc[df_rep_pm["code_postal"].isna(), :].shape # 528 795

(528795, 24)

In [36]:
df_rep_pm.loc[df_rep_pm["code_commune"].isna(), :].shape # 1 441 393

(1441393, 24)

In [29]:
# donc plus de code commune manquants que de code postaux
# on va merger code postal et code_insee
# code_postal = code_postal.loc[:, ["Code_commune_INSEE", "Code_postal"]]

In [39]:
code_postal.columns = [c.lower() for c in code_postal.columns]

In [40]:
#df_rep_pm['code_postal2'] = pd.to_numeric(df_rep_pm['code_postal'], errors='coerce')

In [54]:
# df_rep_pm.loc[df_rep_pm["code_postal2"].isna(),:].shape # 540862 , on a remplacé des valeurs bizarres en valeurs manquantes

(540862, 25)

In [42]:
code_postal.head()

,code_commune_insee,nom_commune,code_postal,ligne_5,libellé_d_acheminement,coordonnees_gps,code_postal2
0,25620,VILLE DU PONT,25650,NaN,VILLE DU PONT,"46.999873398,6.498147193",25650.0
1,25624,VILLERS GRELOT,25640,NaN,VILLERS GRELOT,"47.361512085,6.235167025",25640.0
2,26001,SOLAURE EN DIOIS,26150,AIX EN DIOIS,SOLAURE EN DIOIS,"44.70587354,5.387770455",26150.0
3,26003,ALEYRAC,26770,NaN,ALEYRAC,"44.499150841,4.946083614",26770.0
4,26009,ANDANCETTE,26140,NaN,ANDANCETTE,"45.246625023,4.812717032",26140.0


In [46]:
code_postal["code_departement"] = code_postal["code_commune_insee"].str.slice(start=0,stop=2) # a corriger pour les drom

In [48]:
code_postal[["code_departement", "code_commune_insee"]].head()

,code_departement,code_commune_insee
0,25,25620
1,25,25624
2,26,26001
3,26,26003
4,26,26009


In [52]:
df_rep_pm["code_departement"] = df_rep_pm["code_postal"].astype(str).str.slice(start=0, stop=2)

In [53]:
df_rep_pm[["code_departement", "code_postal"]].head()

,code_departement,code_postal
0,na,NaN
1,na,NaN
2,84,84200.0
3,75,75016.0
4,84,84200.0


In [63]:
code_postal["commune"] = code_postal["nom_commune"].str.lower()

In [64]:
code_postal["commune"] = (code_postal["commune"].str.replace("à", "a")
                                            .str.replace("è", "e")
                                            .str.replace("é", "e")
                                            .str.replace("ç", "c")
                                            .str.replace("ù", "u")
                                            .str.replace("ô", "o"))

In [66]:
code_postal["commune"] = (code_postal["commune"].str.replace("SAINT", "ST"))

In [67]:
code_postal["commune"] = code_postal["commune"].str.replace(" ", "")

In [68]:
code_postal["commune"].value_counts()

livarotpaysdauge     22
stpierreenauge       22
souleuvreenbocage    21
mooreamaiao          20
stpaul               19
                     ..
kirviller             1
manom                 1
fontenaylepesnel      1
kirschlessierck       1
monaco                1
Name: commune, Length: 32710, dtype: int64

In [69]:
code_postal.duplicated(subset=["commune", "code_departement"]).sum() # 4113

4113

In [72]:
code_postal.shape # 39193 -> 35080

(35080, 9)

In [71]:
code_postal = code_postal.drop_duplicates(subset = ["commune", "code_departement"])

In [73]:
code_postal.head(20)

,code_commune_insee,nom_commune,code_postal,ligne_5,libellé_d_acheminement,coordonnees_gps,code_postal2,code_departement,commune
0,25620,VILLE DU PONT,25650,NaN,VILLE DU PONT,"46.999873398,6.498147193",25650.0,25,villedupont
1,25624,VILLERS GRELOT,25640,NaN,VILLERS GRELOT,"47.361512085,6.235167025",25640.0,25,villersgrelot
2,26001,SOLAURE EN DIOIS,26150,AIX EN DIOIS,SOLAURE EN DIOIS,"44.70587354,5.387770455",26150.0,26,solaureendiois
3,26003,ALEYRAC,26770,NaN,ALEYRAC,"44.499150841,4.946083614",26770.0,26,aleyrac
4,26009,ANDANCETTE,26140,NaN,ANDANCETTE,"45.246625023,4.812717032",26140.0,26,andancette
5,26011,AOUSTE SUR SYE,26400,NaN,AOUSTE SUR SYE,"44.706856708,5.057320177",26400.0,26,aoustesursye
6,26013,ARPAVON,26110,NaN,ARPAVON,"44.371127022,5.273184253",26110.0,26,arpavon
7,26021,AUTICHAMP,26400,NaN,AUTICHAMP,"44.683240066,4.972677118",26400.0,26,autichamp
8,26024,BARCELONNE,26120,NaN,BARCELONNE,"44.867074421,5.054652574",26120.0,26,barcelonne
9,26026,BARRET DE LIOURE,26570,NaN,BARRET DE LIOURE,"44.175483103,5.516253783",26570.0,26,barretdelioure


In [74]:
code_postal = code_postal.loc[:, ["code_commune_insee", "code_departement", "commune"]]

In [75]:
df_rep_pm.shape # 9 751 833

(9751833, 28)

In [76]:
df_rep_pm2 = pd.merge(df_rep_pm, code_postal, on=["code_departement", "commune"], how="left")

In [80]:
df_rep_pm2.shape # 9751833

(9751833, 29)

In [81]:
df_rep_pm2.loc[df_rep_pm2["code_commune_insee"].isna(), :].sum()